In [163]:
# overhead

import logging
import math
import random
import numpy as np
import time
import torch as t
import torch.nn as nn
from torch import optim
from torch.nn import utils
import matplotlib.pyplot as plt
from scipy.integrate import odeint

logger = logging.getLogger(__name__)
t.autograd.set_detect_anomaly(True)

In [164]:
# environment parameters

BetaiScaled = 8   # Initial rate constant per infected per day
Betai = BetaiScaled/10000

Gammai = 0.25    # Initial recoveries per person per day
kvi = 0         # Initial vaccination rate

Br  = 500                               # Birthrate (all susceptible)
Muall = 0.1                            # Mortality rate
ts=1


Si = (Gammai + Muall)/Betai          #Initial Susceptible Population
Ii = Br/(Gammai+Muall)-(Muall+kvi)/Betai    #Initial Infected/Infectious Population

Ri = Br/Muall - Si - Ii               #Initial Removed Population
Ni = Ri+Ii+Si                        #Total Population
x0=[Si,Ii]
del_T=1
display(Si)
display(Ii)
display(Betai)
display(kvi)

437.49999999999994

1303.5714285714287

0.0008

0

In [165]:
# define system dynamics
# Notes: 
# 0. You only need to modify the "forward" function
# 1. All variables in "forward" need to be PyTorch tensors.
# 2. All math operations in "forward" has to be differentiable, e.g., default PyTorch functions.
# 3. Do not use inplace operations, e.g., x += 1. Please see the following section for an example that does not work.

class Dynamics(nn.Module):

    def __init__(self):
        super(Dynamics, self).__init__()
    @staticmethod
    def forward(state, action):
        
        

        S=state[0]
        I=state[1]

        Beta=action[0]
        kv=action[1]
        
        # the model equations (see Munz et al. 2009)
        f0 = Br - Beta*S*I - Muall*S-kv*S
        f1 = Beta*S*I - Gammai*I - Muall*I
        display([f0,f1])
        f_vec=t.tensor([f0,f1],requires_grad=True)
        state_k1= state + del_T*f_vec
        state=state_k1
        
        return state

In [166]:
# a deterministic controller
# Note:
# 0. You only need to change the network architecture in "__init__"
# 1. nn.Sigmoid outputs values from 0 to 1, nn.Tanh from -1 to 1
# 2. You have all the freedom to make the network wider (by increasing "dim_hidden") or deeper (by adding more lines to nn.Sequential)
# 3. Always start with something simple

class Controller(nn.Module):

    def __init__(self, dim_input, dim_hidden, dim_output):
        """
        dim_input: # of system states
        dim_output: # of actions
        dim_hidden: up to you
        """
        super(Controller, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(dim_input, dim_hidden),
            nn.Tanh(),
            nn.Linear(dim_hidden, dim_output),
            # You can add more layers here
            nn.Sigmoid()
        )

    def forward(self, state):
        action = self.network(state)
        return action

In [167]:
# the simulator that rolls out x(1), x(2), ..., x(T)
# Note:
# 0. Need to change "initialize_state" to optimize the controller over a distribution of initial states
# 1. self.action_trajectory and self.state_trajectory stores the action and state trajectories along time

class Simulation(nn.Module):

    def __init__(self, controller, dynamics, T):
        super(Simulation, self).__init__()
        self.state = self.initialize_state() 
        self.controller = controller
        self.dynamics = dynamics
        self.T = T
        self.action_trajectory = []
        self.state_trajectory = []

    def forward(self, state):
        self.action_trajectory = []
        self.state_trajectory = []
        for tsamp in range(1,T):
            display(state)
            action = self.controller.forward(state)
            # if tsamp==1:
            #     action=t.tensor([Betai,kvi])
            # display(action)
            state = self.dynamics.forward(state, action)
            # display(state)
            self.action_trajectory.append(action)
            self.state_trajectory.append(state)
        return self.error(state,action)

    @staticmethod
    def initialize_state():
        state = [Si, Ii]  # TODO: need batch of initial states
        return t.tensor(state, requires_grad=False).float()

    def error(self, state, action):
        return (action[0]-Betai)**2 + (state[1]-Ii/2)**2

In [168]:
# set up the optimizer
# Note:
# 0. LBFGS is a good choice if you don't have a large batch size (i.e., a lot of initial states to consider simultaneously)
# 1. You can also try SGD and other momentum-based methods implemented in PyTorch
# 2. You will need to customize "visualize"
# 3. loss.backward is where the gradient is calculated (d_loss/d_variables)
# 4. self.optimizer.step(closure) is where gradient descent is done

class Optimize:
    def __init__(self, simulation):
        self.simulation = simulation
        self.parameters = simulation.controller.parameters()
        self.optimizer = optim.LBFGS(self.parameters, lr=0.01)

    def step(self):
        def closure():
            loss = self.simulation(self.simulation.state)
            self.optimizer.zero_grad()
            loss.backward()
            return loss
        self.optimizer.step(closure)
        return closure()
    
    def train(self, epochs):
        for epoch in range(epochs):
            loss = self.step()
            print('[%d] loss: %.3f' % (epoch + 1, loss))
            self.visualize()

    def visualize(self):
        data = np.array([self.simulation.state_trajectory[i].detach().numpy() for i in range(self.simulation.T)])
        x = data[:, 0]
        y = data[:, 1]
        plt.plot(x, y)
        plt.show()

In [169]:
# Now it's time to run the code!

T = 100  # number of time steps
dim_input = 2  # state space dimensions
dim_hidden = 6  # latent dimensions
dim_output = 2  # action space dimensions
d = Dynamics()  # define dynamics
c = Controller(dim_input, dim_hidden, dim_output)  # define controller
s = Simulation(c, d, T)  # define simulation
o = Optimize(s)  # define optimizer
o.train(40)  # solve the optimization problem

tensor([ 437.5000, 1303.5714])

tensor([0.0008, 0.0000])

[tensor(0.), tensor(0.)]

tensor([ 437.5000, 1303.5714], grad_fn=<AddBackward0>)

tensor([0.3832, 0.6029], grad_fn=<SigmoidBackward0>)

...

tensor([nan, nan], grad_fn=<AddBackward0>)

tensor([nan, nan], grad_fn=<SigmoidBackward0>)

[tensor(nan, grad_fn=<SubBackward0>), tensor(nan, grad_fn=<SubBackward0>)]

c:\Users\baner\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: Error detected in PowBackward0. Traceback of forward call that caused the error:
  File "c:\Users\baner\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\baner\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\baner\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\baner\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 978, in launch_instance
    app.start()
  File "C:\Users\baner\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 712, in start
    self.io_loop.start()
  File "C:\Users\baner\AppData\Roaming\Python\Python310\site-packages\tornado\plat

RuntimeError: Function 'PowBackward0' returned nan values in its 0th output.